# Find frequencies that make only $(\hat{a}^2\hat{b}^2+\hat{a}\hat{b}\hat{d}^2+\hat{d}^4)\hat{c}^\dagger +h.c.$ resonant in the 5th order expansion of $\sin(\hat{a}+\hat{b}+\hat{c}+\hat{d}+h.c.)$

## Here we use linear programming instead of constraint programming and search for any positively-valued frequencies (instead of just integer frequencies).

Import the "Hamiltonian-through-Nonlinearities Engineering" module (it can be installed from PyPI using `pip`).

In [1]:
import hamnonlineng as hnle

Set the letters you want to use for annihilation operators (4 modes in our case).

In [2]:
letters = 'abcd'

Write down (or somehow generate) a list of the monomials that you want to be resonant.

In [3]:
resonant = [hnle.Monomial(1,'aabbC'), # First argument is the constant real factor in front of the operator
            hnle.Monomial(1,'abddC'), # Second argument is the string representing the operators
            hnle.Monomial(1,'Cdddd')]

The convention for typing in or printing out is:
- lower 'a' represents $\hat{a}$
- capital 'A' represents $\hat{a}^\dagger$
- the hermitian conjugate is implicit, i.e. `Monomial(1,'Aab')` is $\hat{a}^\dagger\hat{a}\hat{b}+\hat{a}^\dagger\hat{a}\hat{b}^\dagger$
- the library sorts the expresion to make it "canonical", and given that the presence of a hermitian conjugate is implicit each monomial might print out as its conjugate, i.e. there is no difference between `Monomial(1,'a')` and `Monomial(1,'A')` 

In [4]:
resonant

[1.AABBc, 1.ABcDD, 1.Cdddd]

Now generate the terms that you want to be off resonant: start with the sum $\hat{a}+\hat{b}+\hat{c}+\hat{d}+h.c.$.

In [5]:
op_sum = hnle.operator_sum(letters)
op_sum

1.A
1.B
1.C
1.D

Generate the list of 3rd and 5th order terms in the expansion of $\sin(\hat{a}+\hat{b}+\hat{c}+\hat{d}+h.c.)$.

In [6]:
sine_exp = hnle.sin_terms(op_sum, 3) + hnle.sin_terms(op_sum, 5)
sine_exp_list = sine_exp.m

Filter out of the list:
- terms that match the terms we want to be resonant
- terms that are only annihilation or only creation operators (definitely off-resonant)
- terms that contain only one single mode

In [7]:
off_resonant = hnle.drop_single_mode(
                 hnle.drop_definitely_offresonant(
                   hnle.drop_matching(sine_exp.m, resonant)))
off_resonant = list(off_resonant)

How many terms are left.

In [8]:
len(off_resonant)

365

Finally, solve the constraints:

In [9]:
res = hnle.solve_linearprog_pulp(resonant, off_resonant, letters, maxfreq=20, detune=0.1)

Adding resonant constraints...
Adding off-resonant constraints...
Adding distinct constraints...
Dumping LP file...
Solving...
Status: 1 - Optimal


In [10]:
res

[0.3, 0.7, 2.0, 0.5]